In [12]:
import numpy as np
import pandas as pd

In [13]:
df = pd.read_csv("employee_data.csv", low_memory=False)

In [14]:
df.drop_duplicates('full_name',inplace=True, ignore_index=True)

In [15]:
df.date_hired = pd.to_datetime(df.date_hired, utc=True).dt.strftime('%Y-%m-%d')

In [16]:
df.date_hired = pd.to_datetime(df.date_hired)

In [17]:
df.sort_values(['date_hired','full_name'],inplace=True)
df.reset_index(drop=True,inplace=True)
df['SN'] = df.index + 1
df['unique_id'] = 'TN-' + df['SN'].astype('str') + '-UD'
print(df.unique_id.duplicated().sum())

0


In [18]:
df.duplicated('full_name').sum()

0

In [20]:
df.duplicated('email_address').sum()

0

In [21]:
df.shape

(995, 178)

In [22]:
df.duplicated('unique_id').sum()

0

In [24]:
df.date_hired.isna().sum()

0

In [26]:
df.sort_values(['date_hired','full_name'],inplace=True)

In [31]:
def recalc_att_ytd(dataframe, today): # select sundays on or before sundays
    date_list_column = [] # create list to store dates from date columns
    start_column = dataframe.columns.get_loc('_08_Jan_23') # select index of first date column
    while int(start_column) < dataframe.shape[1]: # store date columns into the list
        date_list_column.append(pd.to_datetime(dataframe.columns[start_column], format='_%d_%b_%y'))
        start_column = int(start_column) + 3
    for i in dataframe.index: # for every person in the table
        date_hired = dataframe.loc[i,'date_hired'] # select date added
        dates_available = [] # create list to store all dates in the table that are after when the user was added to the table and are on or before today
        for date in date_list_column: # for every date present in the original table
            if date >= date_hired and date <= pd.to_datetime(today): # check if the date is on or after the user was added and on or before today
                column_name = date.strftime('_%d_%b_%y')
                dates_available.append(column_name) # store that date into a list
        sundays_by_today = dataframe.loc[i,dates_available] # create dataframe with only those date columns
        sundays_by_today.fillna('',inplace=True)
        all_count = sundays_by_today.count() # count total days
        try:
            present_count = sundays_by_today.value_counts()['Present'] # count days present
            dataframe.loc[i,'att_ytd'] = round((present_count/all_count*100),2) # recalculate att_ytd
        except: dataframe.loc[i,'att_ytd'] = 0.00
    return dataframe

In [32]:
from datetime import date
today = date.today()

In [33]:
df = recalc_att_ytd(df,today)

In [34]:
df.att_ytd.value_counts(dropna=False)

52.94    148
41.18    125
47.06    122
58.82    106
35.29     81
        ... 
72.73      1
18.18      1
27.27      1
58.33      1
71.43      1
Name: att_ytd, Length: 63, dtype: int64

In [35]:
df.dtypes

first_name                     object
last_name                      object
full_name                      object
phone_number                    int64
email_address                  object
                               ...   
comment24122023               float64
checkin_location_24_Dec_23    float64
_31_Dec_23                    float64
comment31122023               float64
checkin_location_31_Dec_23    float64
Length: 178, dtype: object

In [37]:
df.to_csv('employee_data.csv',index=False)